In [31]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import pandas as pd

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.2.0


In [32]:
BATCH_SIZE = 600
## Importing a local version of input_data

In [33]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import warnings

# load the image
img = load_img("D:\\Practice\\Computer Vision - Vehical Classification\\train\\images\\0.jpg")
# report details about the image
print(type(img))
print(img.format)
print(img.mode)
print(img.size)
# show the image
img_np = img_to_array(img)
img_np.shape
img_np = np.expand_dims(img_np, 0)
print(img_np.shape)
img = load_img("D:\\Practice\\Computer Vision - Vehical Classification\\train\\images\\1.jpg")
img_np = np.append(img_np ,img_to_array(img))
img_np.shape

<class 'PIL.JpegImagePlugin.JpegImageFile'>
JPEG
RGB
(224, 224)
(1, 224, 224, 3)


(301056,)

In [34]:
#filelist = [ str(i)+'.jpg'for i in range(0,2351)]
trainX = pd.read_csv("D:\\Practice\\Computer Vision - Vehical Classification\\train\\train.csv")
filelist = trainX['image_names'].values
trainY = trainX['emergency_or_not'].values
trainX = np.array([np.array(img_to_array(load_img("D:\\Practice\\Computer Vision - Vehical Classification\\train\\images\\"+fname))) for fname in filelist])
testX = pd.read_csv("D:\\Practice\\Computer Vision - Vehical Classification\\test.csv")
filelist = testX['image_names'].values
testX = np.array([np.array(img_to_array(load_img("D:\\Practice\\Computer Vision - Vehical Classification\\train\\images\\"+fname))) for fname in filelist])
print(trainX.shape,testX.shape,trainY.shape)

(1646, 224, 224, 3) (706, 224, 224, 3) (1646,)


In [35]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

class MiniVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1
        # first CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
		# second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))
		# return the constructed network architecture
		return model

In [36]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
## from pyimagesearch.minivggnet import MiniVGGNet
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from keras.datasets import fashion_mnist
from keras.utils import np_utils
from keras import backend as K
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import cv2
# initialize the number of epochs to train for, base learning rate,
# and batch size
NUM_EPOCHS = 7
INIT_LR = 1e-2
BS = 32
trainY = trainY.reshape(-1,1)
print(trainX.shape, testX.shape, trainY.shape)

(1646, 224, 224, 3) (706, 224, 224, 3) (1646, 1)


In [37]:
type(trainY)

numpy.ndarray

In [38]:
# scale data to the range of [0, 1]
# trainX = trainX.astype("float32") / 255.0
# testX = testX.astype("float32") / 255.0
# one-hot encode the training and testing labels
 x= np_utils.to_categorical(trainY, 2)
# testY = np_utils.to_categorical(testY, 10)
# initialize the label names
# labelNames = ["top", "trouser", "pullover", "dress", "coat","sandal", "shirt", "sneaker", "bag", "ankle boot"]
x

In [40]:
trainY = x

In [41]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)
model = MiniVGGNet.build(width=224, height=224, depth=3, classes=2)
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])
# train the network
print("[INFO] training model...")
# load model
from keras.models import load_model
model_name = "model_"+str(NUM_EPOCHS)+".h5"
try :
    model = load_model(model_name)

except :
    print("No File Found")
created_new_model = False

if(model.name != model_name):
    created_new_model = True
    print("I should be skipping if I already have a model for this ... ")
    H = model.fit(trainX, trainY,validation_data=(trainX, trainY),batch_size=BS, epochs=NUM_EPOCHS)

[INFO] compiling model...
[INFO] training model...
No File Found
I should be skipping if I already have a model for this ... 
Train on 1646 samples, validate on 1646 samples
Epoch 1/20
1646/1646 [==============================] - 361s 220ms/step - loss: 0.8523 - accuracy: 0.6665 - val_loss: 11.9621 - val_accuracy: 0.5863
Epoch 2/20
1646/1646 [==============================] - 364s 221ms/step - loss: 0.5938 - accuracy: 0.7776 - val_loss: 22.4028 - val_accuracy: 0.5930
Epoch 3/20
1646/1646 [==============================] - 358s 217ms/step - loss: 0.6197 - accuracy: 0.8074 - val_loss: 1.9661 - val_accuracy: 0.6537
Epoch 4/20
1646/1646 [==============================] - 366s 222ms/step - loss: 0.4070 - accuracy: 0.8597 - val_loss: 0.3403 - val_accuracy: 0.8937
Epoch 5/20
1646/1646 [==============================] - 355s 215ms/step - loss: 0.3426 - accuracy: 0.8852 - val_loss: 0.1263 - val_accuracy: 0.9435
Epoch 6/20
1646/1646 [==============================] - 361s 220ms/step - loss: 0.24

In [42]:
if(created_new_model):
    # save model and architecture to single file
    model_name = "model_"+str(NUM_EPOCHS)+".h5"
    model.name = model_name
    model.save(model_name)
    print("Saved model to disk")
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(H.history) 

   # or save to csv: 
    hist_csv_file = 'history.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)
        
    print("Saved new model and History")
        
if(not created_new_model) :
    hist_df = pd.read_csv('history.csv')
    H.history = hist_df.to_dict()
    print("Loaded History")

Saved model to disk
Saved new model and History


In [54]:
y_pred = model.predict_classes(testX)
submit = pd.read_csv('sample_submission.csv')
submit.emergency_or_not = pd.Series(y_pred)
submit.head()
submit.to_csv("results.csv",index=False)
print("Successfully created results")

Successfully created results


In [45]:
%matplotlib inline
# make predictions on the test set
preds = model.predict(testX)
# show a nicely formatted classification report
print("[INFO] evaluating network...")
print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1),
	target_names=labelNames))
# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")
plt.show()

[INFO] evaluating network...


NameError: name 'testY' is not defined

In [57]:
y_pred_prob = model.predict(testX)
y_pred_prob

array([[9.9949861e-01, 5.0136726e-04],
       [7.0315524e-05, 9.9992967e-01],
       [9.9999917e-01, 8.0075023e-07],
       ...,
       [4.2030779e-06, 9.9999583e-01],
       [9.9990427e-01, 9.5689815e-05],
       [9.9997902e-01, 2.0953301e-05]], dtype=float32)

In [63]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
type(y_pred_prob)
y_pred_prob

array([[0.999, 0.001],
       [0.000, 1.000],
       [1.000, 0.000],
       ...,
       [0.000, 1.000],
       [1.000, 0.000],
       [1.000, 0.000]], dtype=float32)

array([[0.000, 1.000],
       [1.000, 0.000],
       [0.003, 0.997]], dtype=float32)